### Temporary file used to debug super slow TF-IDF implementation

In [ ]:
import pandas as pd
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from datasets import load_dataset
import datasets
import torch
from collections import Counter

# import string library function  
import string

In [ ]:
#This notebook was used to fix our tf-idf implementation.
# Made it vectorized, we're good!

In [ ]:
dataset_dict = datasets.load_from_disk("./Dataset/CodeSearchCorpus/")
train_dataset = dataset_dict["train"]
np.random.seed(1)
train_subset_indices = np.random.choice(len(train_dataset), 10000, replace = False)
train_dataset_subset = train_dataset.select(train_subset_indices)


In [ ]:
# train_dataset_subset[0]["func_code_tokens"]
tsed_DF = train_dataset_subset.to_pandas() #train-subset-embeddings-dataset_DF
def clean_code_tokens(lst):
    result = string.punctuation 
    new_lst = [] 
    for character in lst:
        if character in result:
            continue
        else:
            new_lst.append(character)
    return new_lst

tsed_DF["clean_code_tokens"] =  tsed_DF["func_code_tokens"].apply(clean_code_tokens)


In [ ]:
documents = tsed_DF["clean_code_tokens"].to_dict()

all_words = []
for i in list(tsed_DF["clean_code_tokens"].to_dict().values()):
    all_words += i

all_words = list(set(all_words))
all_words


try:
     with open('./pickleObjects/inverted_index.pkl', 'rb') as f:
        inverted_index = pickle.load(f) # deserialize using load()
        f.close()
except:
    inverted_index = {}

    for word in all_words:
            if word != "":
                lst_docs = []
                for i, doc in documents.items():
                    if word in doc:
                        lst_docs.append(i)
            
                inverted_index[word] = lst_docs

In [ ]:
len(all_words) == len(inverted_index)

In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = inverted_index[word]
    except:
        pass

    if type(c) == list:
        return len(c)
    else:
        return 0



N = 10000 #number of rows sampled
tf_idf = {}
for i in range(N):
    # print(i)
    tokens = tsed_DF["clean_code_tokens"].iloc[i]
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token] / words_count
        df = doc_freq(token)
        idf = np.log((N + 1) / (df + 1))

        tf_idf[i, token] = tf * idf
    

In [ ]:
tsed_DF["clean_code_tokens"].iloc[1]

In [ ]:
# inverted_index
tf_idf

In [ ]:
len(tf_idf)

In [ ]:
all_words
all_words_dict = dict(zip(all_words, range(len(all_words))))

In [ ]:
all_words_dict

In [ ]:
#Vectorising tf-idf

D = np.zeros((N, len(all_words)))

for i in tf_idf:
    try:
        ind = all_words_dict[i[1]]
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [ ]:
D[2].sum()

In [ ]:
np.zeros((N, len(all_words))).shape

In [ ]:
def gen_vector(tokens):

    Q = np.zeros((len(all_words)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))

        try:
            ind = all_words_dict[token]
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
gen_vector("pandas how to select first 10 rows".split()).sum()